In [ ]:
import sys
sys.path.append('..')

import os
import math
import torch
import random
import numpy as np
from pathlib import Path
from torch.cuda import empty_cache
from IPython.display import Audio
import matplotlib.pyplot as plt

from configs.commons import Config as cfg
from configs.commons import DEVICE, CACHE_DIR, CTX
from configs.constants import *

from omni.hfload import convert_to_hf
from datalib.tokenlib import get_tokenizer

In [ ]:
DEVICE = 'cuda:0'

In [ ]:
omni_model = convert_to_hf(path=f'../omni_task.pt', device=DEVICE)
semantic_acoustic_model = convert_to_hf(path=f'../semantic_acoustic.pt', device=DEVICE)

text_tokenizer = get_tokenizer(TEXT, device='cpu')
acoustic_tokenizer = get_tokenizer(ACOUSTIC, device=DEVICE)

In [ ]:
convert_token = text_tokenizer.encode(cfg.TASK_TOKENS[CONVERT])
continue_token = text_tokenizer.encode(cfg.TASK_TOKENS[CONTINUE])
stop_token = text_tokenizer.encode(cfg.STOP_TOKEN)
semantic_modality_token = text_tokenizer.encode(cfg.MODALITY_TOKENS[SEMANTIC])
acoustic_modality_token = text_tokenizer.encode(cfg.MODALITY_TOKENS[ACOUSTIC])
text_modality_token = text_tokenizer.encode(cfg.MODALITY_TOKENS[TEXT])
speaker_id = text_tokenizer.encode("[spkr_unk]")

omni_model.generation_config.eos_token_id = stop_token
semantic_acoustic_model.generation_config.eos_token_id = stop_token

In [ ]:
random_txt = "once upon a time, in a cozy little house at the edge of a forest, lived a curious mouse named pip."
txt_toks = np.array(text_tokenizer.encode(random_txt))

In [ ]:
input_tokens = np.hstack([
    text_modality_token,
    txt_toks,
    continue_token,
    semantic_modality_token
])
input_tokens = (torch.tensor(input_tokens, dtype=torch.long, device=DEVICE)[None, ...])
print(f'Text tokens: {input_tokens.shape}')

In [ ]:
with CTX:
    semantic_tokens = omni_model.generate(
        input_tokens,
        max_length=1024,
        temperature=0.7,
        top_k=100,
        do_sample=True
    )
    semantic_tokens = semantic_tokens.detach().cpu().numpy()[0]
    semantic_tokens = semantic_tokens[input_tokens.shape[-1]:]
    print(semantic_tokens.shape)

In [ ]:
semantic_tokens

In [ ]:
text_tokenizer.decode(semantic_tokens)

In [ ]:
np.where(semantic_tokens == 58)

In [ ]:
# end_idx = np.where(semantic_tokens == stop_token)[0][0]
end_idx = 126
semantic_tokens = semantic_tokens[0:end_idx]

In [ ]:
semantic_tokens = np.hstack([
    semantic_modality_token,
    semantic_tokens,
    convert_token,
    speaker_id,
    acoustic_modality_token
])
semantic_tokens = (torch.tensor(semantic_tokens, dtype=torch.long, device=DEVICE)[None, ...])
print(f'Semantic tokens: {semantic_tokens.shape}')

In [ ]:
with CTX:
    acoustic_tokens = semantic_acoustic_model.generate(
        semantic_tokens,
        max_length=3072,
        temperature=0.9,
        top_k=100,
        do_sample=True
    )

    acoustic_tokens = acoustic_tokens.detach().cpu().numpy()[0]
    acoustic_tokens = acoustic_tokens[semantic_tokens.shape[-1]:]
    print(acoustic_tokens.shape)

end_idx = np.where(acoustic_tokens == stop_token)[0][0]
acoustic_tokens = acoustic_tokens[0:end_idx]
acoustic_tokens = acoustic_tokens - cfg.OFFSET[ACOUSTIC]

if len(acoustic_tokens) % 2 == 1:
    acoustic_tokens = acoustic_tokens[:-1]

print(f'Acoustic tokens: {acoustic_tokens.shape}')

In [ ]:
wav = acoustic_tokenizer.decode(torch.tensor(acoustic_tokens))
wav = wav[0].cpu().numpy()
Audio(wav, rate=24000)

In [ ]:
## Testing with custom tokens
prompt = np.load('../prompts/lj_female_long/tokens.npz')
temp_sem_toks = prompt['SEMANTIC']
temp_sem_toks += cfg.OFFSET[SEMANTIC]
temp_sem_toks = np.hstack([
    semantic_modality_token,
    temp_sem_toks,
    convert_token,
    speaker_id,
    acoustic_modality_token
])
temp_sem_toks = (torch.tensor(temp_sem_toks, dtype=torch.long, device=DEVICE)[None, ...])
print(temp_sem_toks.shape)